<a href="https://colab.research.google.com/github/MahdiTheGreat/Diagnostic-Systems/blob/main/DAIS_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MahdiTheGreat/Diagnostic-Systems.git
%cd Diagnostic-Systems
%ls

Cloning into 'Diagnostic-Systems'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 73.97 KiB | 18.49 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/Diagnostic-Systems
README.md  wdbc.pkl


In [2]:
import pickle
import pandas as pd

# Load the pickle file
df = pd.read_pickle('wdbc.pkl')

# Display the DataFrame
df[0:10]



,id,malignant,radius_0,texture_0,perimeter_0,area_0,smoothness_0,compactness_0,concavity_0,concave points_0,...,radius_2,texture_2,perimeter_2,area_2,smoothness_2,compactness_2,concavity_2,concave points_2,symmetry_2,fractal dimension_2
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,843786,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,844359,1,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,84458202,1,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,844981,1,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,84501001,1,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.8 MB/s eta 0:00:00


In [4]:

# Basic statistics
print(df['malignant'].describe())

# Value counts (for categorical data)
print(df['malignant'].value_counts())


count    569.000000
mean       0.372583
std        0.483918
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: malignant, dtype: float64
malignant
0    357
1    212
Name: count, dtype: int64


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import optuna

# Load your dataset
df = pd.read_pickle('wdbc.pkl')
X = df.drop('malignant', axis=1)  # Adjust 'target' to your actual target column name
y = df['malignant']  # Adjust 'target' to your actual target column name

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    # Define the hyperparameters to tune via Optuna
    n_estimators = trial.suggest_int('n_estimators', 50, 400)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50)
    max_features = trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])

    # Create the random forest classifier
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # Fit the model
    clf.fit(X_train, y_train)

    # Evaluate the model
    y_pred = clf.predict(X_test)
    f1_score_value = f1_score(y_test, y_pred)

    return f1_score_value

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best f1_score_value:', study.best_trial.value)

# Best model parameters
best_params = study.best_trial.params
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)


[I 2025-02-14 11:58:44,087] A new study created in memory with name: no-name-9c5696b2-5b59-4a7f-9fb7-6c3335d71dd3
[I 2025-02-14 11:58:46,804] Trial 0 finished with value: 0.9411764705882353 and parameters: {'n_estimators': 350, 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-02-14 11:58:47,442] Trial 1 finished with value: 0.9382716049382716 and parameters: {'n_estimators': 184, 'max_depth': 10, 'min_samples_split': 17, 'min_samples_leaf': 49, 'max_features': 'log2'}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-02-14 11:58:47,898] Trial 2 finished with value: 0.9397590361445783 and parameters: {'n_estimators': 100, 'max_depth': 2, 'min_samples_split': 22, 'min_samples_leaf': 18, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-02-14 11:58:50,581] Trial 3 finished with value: 0.9411764705882353 and parameters: {'n_estimators': 336, 'max_depth': 11, 'm

Number of finished trials: 100
Best trial: {'n_estimators': 220, 'max_depth': 12, 'min_samples_split': 30, 'min_samples_leaf': 40, 'max_features': 'log2'}
Best f1_score_value: 0.963855421686747


RandomForestClassifier(max_depth=12, max_features='log2', min_samples_leaf=40,
                       min_samples_split=30, n_estimators=220)

In [6]:
display(df[0:1])
display(df.columns)

,id,malignant,radius_0,texture_0,perimeter_0,area_0,smoothness_0,compactness_0,concavity_0,concave points_0,...,radius_2,texture_2,perimeter_2,area_2,smoothness_2,compactness_2,concavity_2,concave points_2,symmetry_2,fractal dimension_2
0,842302,1,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189


Index(['id', 'malignant', 'radius_0', 'texture_0', 'perimeter_0', 'area_0',
       'smoothness_0', 'compactness_0', 'concavity_0', 'concave points_0',
       'symmetry_0', 'fractal dimension_0', 'radius_1', 'texture_1',
       'perimeter_1', 'area_1', 'smoothness_1', 'compactness_1', 'concavity_1',
       'concave points_1', 'symmetry_1', 'fractal dimension_1', 'radius_2',
       'texture_2', 'perimeter_2', 'area_2', 'smoothness_2', 'compactness_2',
       'concavity_2', 'concave points_2', 'symmetry_2', 'fractal dimension_2'],
      dtype='object')

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import optuna


# Initialize models
models = {
    "Random Forest": RandomForestClassifier(**best_params),  # assuming best_params are known from previous optimization
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),  # Increase max_iter if convergence issue
}

# Fit models and evaluate
results = {}
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred)
    results[name] = score
    print(f"{name} f1 score: {score:.2f}")

Training Random Forest...
Random Forest f1 score: 0.95
Training Decision Tree...
Decision Tree f1 score: 0.92
Training Logistic Regression...
Logistic Regression f1 score: 0.91


In [ ]:
model=SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = f1_score(y_test, y_pred)
print(f"{name} f1 score: {score:.2f}")
